In [17]:
import os
from pathlib import Path
import glob
import re
import shutil
from random import shuffle

import torch
from torchvision.io import decode_image, write_jpeg
from natsort import natsorted
from tqdm import tqdm

In [18]:
ORIGINAL_DATASET = os.path.join(os.sep, 'ml', 'LIZA_dataset', 'original_split', '**')
DATASET = os.path.join(os.sep, 'ml', 'LIZA_dataset', 'classifier')
HUMAN = os.path.join(os.sep, 'ml', 'LIZA_dataset', 'classifier', 'human')
NOT_HUMAN = os.path.join(os.sep, 'ml', 'LIZA_dataset', 'classifier', 'not-human')

if os.path.isdir(DATASET):
    shutil.rmtree(DATASET)
[os.mkdir(dir) for dir in (DATASET, HUMAN, NOT_HUMAN)]

AREA_THRESHOLD = 2800

In [19]:
images = [file for file in glob.glob(os.path.join(ORIGINAL_DATASET, '*'), recursive=True) if re.match(r'(.*\.jpg)', file)]
annotations = [file for file in glob.glob(os.path.join(ORIGINAL_DATASET, '*'), recursive=True) if re.match(r'(.*\.txt)', file)]

images = natsorted(images)
annotations = natsorted(annotations)

len(images), len(annotations)

(2166633, 2168989)

In [20]:
for image, annotation in tqdm(zip(images, annotations)):
    if Path(image).stem != Path(annotation).stem:
        raise ValueError('Images and annotations indices do not match')
    stem = Path(image).stem

    T = decode_image(image)
    _, height, width = T.shape

    
    with open(annotation) as file:
        labels = file.read().splitlines()

    bboxes = torch.empty((len(labels), 4), dtype=torch.float32)
    for i in range(len(labels)):
        if not labels[i]:
            continue
            
        _, bbox_x, bbox_y, bbox_w, bbox_h = labels[i].split(' ')
        bbox_x, bbox_y, bbox_w, bbox_h = list(map(float, (bbox_x, bbox_y, bbox_w, bbox_h)))
        
        bbox_l = bbox_x - bbox_w / 2  # from center coordinate to left
        bbox_t = bbox_y - bbox_h / 2  # from center coordinate to top
        bbox = torch.as_tensor((bbox_l * width, bbox_t * height, bbox_w * width, bbox_h * height), dtype=torch.int32)
        if bbox[2] <= 0 or bbox[3] <= 0 or bbox[2]*bbox[3] <= AREA_THRESHOLD:
            continue
            
        bboxes[i] = bbox
        
        path = os.path.join(HUMAN, f'{stem}_{i}.jpg')
        
        if bbox[0] and bbox[1]:
            patch = T[:, bbox[1]:bbox[1] + bbox[3], bbox[0]:bbox[0] + bbox[2]]
        elif bbox[0]:
            patch = T[:, :bbox[3], 0:bbox[0] + bbox[2]]
        elif bbox[1]:
            patch = T[:, bbox[1]:bbox[1] + bbox[3], :bbox[2]]
        else:
            patch = T[:, :bbox[3], :bbox[2]]
            
        write_jpeg(patch, path)

    # predict and stuff

8813it [00:19, 462.80it/s]


KeyboardInterrupt: 